In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from nb_005 import *
from collections import Counter

# Wikitext 2

## Data

Download the dataset [here](https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-v1.zip) and unzip it so it's in the folder wikitext.

In [13]:
EOS = '<eos>'
PATH=Path('data/wikitext')

Small helper function to read the tokens.

In [14]:
def read_file(filename):
    tokens = []
    with open(PATH/filename, encoding='utf8') as f:
        for line in f:
            tokens.append(line.split() + [EOS])
    return np.array(tokens)

In [15]:
train_tok = read_file('wiki.train.tokens')
valid_tok = read_file('wiki.valid.tokens')
test_tok = read_file('wiki.test.tokens')

In [16]:
len(train_tok), len(valid_tok), len(test_tok)

(36718, 3760, 4358)

In [17]:
' '.join(train_tok[4][:20])

'The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II'

In [18]:
cnt = Counter(word for sent in train_tok for word in sent)
cnt.most_common(10)

[('the', 113161),
 (',', 99913),
 ('.', 73388),
 ('of', 56889),
 ('<unk>', 54625),
 ('and', 50603),
 ('in', 39453),
 ('to', 39190),
 ('<eos>', 36718),
 ('a', 34237)]

Give an id to each token and add the pad token (just in case we need it).

In [19]:
itos = [o for o,c in cnt.most_common()]
itos.insert(0,'<pad>')

In [20]:
vocab_size = len(itos); vocab_size

33279

Creates the mapping from token to id then numericalizing our datasets.

In [21]:
stoi = collections.defaultdict(lambda : 5, {w:i for i,w in enumerate(itos)})

In [22]:
train_ids = np.array([([stoi[w] for w in s]) for s in train_tok])
valid_ids = np.array([([stoi[w] for w in s]) for s in valid_tok])
test_ids = np.array([([stoi[w] for w in s]) for s in test_tok])

In [23]:
class LanguageModelLoader():
    """ Returns a language model iterator that iterates through batches that are of length N(bptt,5)
    The first batch returned is always bptt+25; the max possible width.  This is done because of they way that pytorch
    allocates cuda memory in order to prevent multiple buffers from being created as the batch width grows.
    """
    def __init__(self, nums, bs, bptt, backwards=False):
        self.bs,self.bptt,self.backwards = bs,bptt,backwards
        self.data = self.batchify(nums)
        self.i,self.iter = 0,0
        self.n = len(self.data)

    def __iter__(self):
        self.i,self.iter = 0,0
        while self.i < self.n-1 and self.iter<len(self):
            if self.i == 0:
                seq_len = self.bptt + 5 * 5
            else:
                bptt = self.bptt if np.random.random() < 0.95 else self.bptt / 2.
                seq_len = max(5, int(np.random.normal(bptt, 5)))
            res = self.get_batch(self.i, seq_len)
            self.i += seq_len
            self.iter += 1
            yield res

    def __len__(self): return self.n // self.bptt - 1

    def batchify(self, data):
        nb = data.shape[0] // self.bs
        data = np.array(data[:nb*self.bs])
        data = data.reshape(self.bs, -1).T
        if self.backwards: data=data[::-1]
        return LongTensor(data)

    def get_batch(self, i, seq_len):
        source = self.data
        seq_len = min(seq_len, len(source) - 1 - i)
        return source[i:i+seq_len], source[i+1:i+1+seq_len].contiguous().view(-1)

In [24]:
bs,bptt = 100,70
train_dl = LanguageModelLoader(np.concatenate(train_ids), bs, bptt)
valid_dl = LanguageModelLoader(np.concatenate(valid_ids), bs, bptt)

In [25]:
class LMDataBunch():
    def __init__(self, train_dl, valid_dl, bs=64, device=None):
        self.device = default_device if device is None else device
        self.train_dl = DeviceDataLoader(train_dl, self.device, progress_func=tqdm)
        self.valid_dl = DeviceDataLoader(valid_dl, self.device, progress_func=tqdm)

    @property
    def train_ds(self): return self.train_dl.dl.dataset
    @property
    def valid_ds(self): return self.valid_dl.dl.dataset

In [26]:
data = LMDataBunch(train_dl, valid_dl, bs)

## Model

### 1. Dropout

We want to use the AWD-LSTM from [Stephen Merity](https://arxiv.org/abs/1708.02182). First, we'll need all different kinds of dropouts. Dropout consists into replacing some coefficients by 0 with probability p. To ensure that the averga of the weights remains constant, we apply a correction to the weights that aren't nullified of a factor `1/(1-p)`.

In [27]:
def dropout_mask(x, sz, p):
    "Returns a dropout mask of the same type as x, size sz, with probability p to cancel an element."
    return x.new(*sz).bernoulli_(1-p)/(1-p)

In [28]:
x = torch.randn(10,10)
dropout_mask(x, (10,10), 0.5)

tensor([[2., 0., 0., 2., 0., 2., 0., 0., 2., 2.],
        [2., 2., 2., 0., 2., 0., 0., 2., 0., 2.],
        [2., 2., 0., 2., 2., 0., 2., 0., 2., 2.],
        [2., 0., 2., 0., 2., 0., 0., 0., 0., 0.],
        [2., 0., 2., 0., 2., 0., 0., 2., 2., 0.],
        [2., 0., 2., 0., 0., 0., 0., 2., 0., 0.],
        [2., 0., 0., 0., 0., 0., 0., 0., 2., 2.],
        [2., 2., 0., 2., 0., 2., 0., 0., 2., 0.],
        [2., 0., 0., 2., 2., 0., 2., 0., 0., 0.],
        [2., 2., 2., 0., 2., 0., 0., 0., 0., 0.]])

Once with have a dropout mask `m`, applying the dropout to `x` is simply done by `x = x * m`. We create our own dropout mask and don't rely on pytorch dropout because we want to nullify the coefficients on the batch dimension but not the token dimension (aka the same coefficients are replaced by zero for each word in the sentence). 

Inside a RNN, a tensor x will have three dimensions: seq_len, bs, vocab_size, so we create a dropout mask for the last two dimensions and broadcast it to the first dimension.

In [29]:
class RNNDropout(nn.Module):
    def __init__(self, p=0.5):
        super().__init__()
        self.p=p

    def forward(self, x):
        if not self.training or not self.p: return x
        m = dropout_mask(x.data, (1, x.size(1), x.size(2)), self.p)
        return m * x

In [30]:
dp_test = RNNDropout(0.5)
x = torch.randn(2,5,10)
x, dp_test(x)

(tensor([[[-1.6119,  0.9999,  0.4388,  0.2222, -1.3747, -0.2785,  1.2303,
           -0.1411,  0.7163,  0.2330],
          [ 0.7123, -1.7036, -1.0978,  0.2369, -0.2148,  0.2562,  0.7597,
            1.1495,  1.4098, -0.3210],
          [ 0.2747, -1.0826, -0.3270,  1.2940,  0.4235, -1.5832, -2.4821,
           -1.6879,  0.4896,  1.3344],
          [ 0.6158, -1.1154, -0.4765, -0.0512,  1.7400,  0.2477,  0.3752,
            0.6315,  0.8837, -0.1232],
          [-1.9605, -0.4867,  0.1933, -0.7489,  0.3968,  1.2791, -0.4492,
            0.4778, -1.7446, -0.2644]],
 
         [[-1.4188,  0.0944, -0.2581, -0.2301,  0.2060, -0.6339, -0.1143,
           -0.1516,  0.8578,  0.3738],
          [-0.7394, -0.7996,  3.0573,  1.0905, -0.7686, -0.1423, -1.0784,
           -0.1240, -0.1343, -0.4616],
          [ 1.2578,  0.1533, -2.2479, -0.5701,  0.0440, -0.6105, -0.9318,
           -0.7878,  0.8716,  1.2730],
          [ 0.7478, -1.1006,  0.1173,  0.4631,  2.1140, -0.2260, -0.4207,
           -0.3985,

In [31]:
class WeightDropout(nn.Module):
    "A module that warps another layer in which some weights will be replaced by 0 during training."
    
    def __init__(self, module, dropout, layer_names=['weight_hh_l0']):
        super().__init__()
        self.module,self.dropout,self.layer_names = module,dropout,layer_names
        for layer in self.layer_names:
            #Makes a copy of the weights of the selected layers.
            w = getattr(self.module, layer)
            self.register_parameter(f'{layer}_raw', nn.Parameter(w.data))
    
    def _setweights(self):
        for layer in self.layer_names:
            raw_w = getattr(self, f'{layer}_raw')
            self.module._parameters[layer] = F.dropout(raw_w, p=self.dropout, training=self.training)
            
    def forward(self, *args):
        self._setweights()
        return self.module.forward(*args)
    
    def reset(self):
        if hasattr(self.module, 'reset'): self.module.reset()

In [32]:
module = nn.LSTM(20, 20)
dp_module = WeightDropout(module, 0.5)
opt = optim.SGD(dp_module.parameters(), 10)
dp_module.train()

WeightDropout(
  (module): LSTM(20, 20)
)

In [33]:
x = torch.randn(2,5,20)
x.requires_grad_(requires_grad=True)
h = (torch.zeros(1,5,20), torch.zeros(1,5,20))
for _ in range(5): x,h = dp_module(x,h)

In [34]:
getattr(dp_module.module, 'weight_hh_l0'),getattr(dp_module,'weight_hh_l0_raw')

(tensor([[-0.2556, -0.0587, -0.0000,  ..., -0.2571, -0.0000, -0.3765],
         [ 0.0000, -0.0000, -0.1356,  ...,  0.0000,  0.0000,  0.4272],
         [ 0.1696,  0.0000, -0.0742,  ..., -0.0000, -0.0000, -0.0785],
         ...,
         [ 0.3813,  0.0000, -0.0000,  ..., -0.0000,  0.0000, -0.0000],
         [ 0.1976,  0.0000, -0.3747,  ...,  0.3374, -0.0000,  0.0574],
         [-0.1834,  0.4082,  0.2302,  ..., -0.1753,  0.0000,  0.0577]],
        grad_fn=<DropoutBackward>), Parameter containing:
 tensor([[-0.1278, -0.0293, -0.1968,  ..., -0.1285, -0.1741, -0.1883],
         [ 0.0693, -0.1413, -0.0678,  ...,  0.0508,  0.1251,  0.2136],
         [ 0.0848,  0.2201, -0.0371,  ..., -0.1575, -0.1660, -0.0392],
         ...,
         [ 0.1907,  0.1187, -0.2121,  ..., -0.0253,  0.2213, -0.1784],
         [ 0.0988,  0.1363, -0.1874,  ...,  0.1687, -0.0992,  0.0287],
         [-0.0917,  0.2041,  0.1151,  ..., -0.0876,  0.0616,  0.0288]],
        requires_grad=True))

In [35]:
target = torch.randint(0,20,(10,)).long()
loss = F.nll_loss(x.view(-1,20), target)
loss.backward()
opt.step()

In [36]:
w, w_raw = getattr(dp_module.module, 'weight_hh_l0'),getattr(dp_module,'weight_hh_l0_raw')
w.grad, w_raw.grad

(None, tensor([[-0.0004,  0.0002,  0.0003,  ..., -0.0002,  0.0000,  0.0000],
         [ 0.0001, -0.0002, -0.0001,  ...,  0.0003, -0.0000,  0.0000],
         [-0.0001, -0.0000, -0.0000,  ..., -0.0000, -0.0000,  0.0000],
         ...,
         [-0.0000,  0.0001, -0.0000,  ..., -0.0001,  0.0000,  0.0000],
         [ 0.0000,  0.0000, -0.0000,  ...,  0.0000, -0.0000,  0.0000],
         [-0.0001, -0.0000, -0.0000,  ..., -0.0000,  0.0000, -0.0001]]))

In [37]:
getattr(dp_module.module, 'weight_hh_l0'),getattr(dp_module,'weight_hh_l0_raw')

(tensor([[-0.2556, -0.0587, -0.0000,  ..., -0.2571, -0.0000, -0.3765],
         [ 0.0000, -0.0000, -0.1356,  ...,  0.0000,  0.0000,  0.4272],
         [ 0.1696,  0.0000, -0.0742,  ..., -0.0000, -0.0000, -0.0785],
         ...,
         [ 0.3813,  0.0000, -0.0000,  ..., -0.0000,  0.0000, -0.0000],
         [ 0.1976,  0.0000, -0.3747,  ...,  0.3374, -0.0000,  0.0574],
         [-0.1834,  0.4082,  0.2302,  ..., -0.1753,  0.0000,  0.0577]],
        grad_fn=<DropoutBackward>), Parameter containing:
 tensor([[-0.1234, -0.0309, -0.1996,  ..., -0.1261, -0.1741, -0.1883],
         [ 0.0684, -0.1391, -0.0665,  ...,  0.0476,  0.1251,  0.2134],
         [ 0.0854,  0.2203, -0.0367,  ..., -0.1574, -0.1658, -0.0394],
         ...,
         [ 0.1910,  0.1180, -0.2121,  ..., -0.0241,  0.2208, -0.1784],
         [ 0.0987,  0.1362, -0.1873,  ...,  0.1684, -0.0991,  0.0284],
         [-0.0911,  0.2044,  0.1152,  ..., -0.0874,  0.0616,  0.0295]],
        requires_grad=True))

In [38]:
class EmbeddingDropout(nn.Module):

    "Applies dropout in the embedding layer by zeroing out some elements of the embedding vector."
    def __init__(self, emb):
        super().__init__()
        self.emb = emb
        self.pad_idx = self.emb.padding_idx
        if self.pad_idx is None: self.pad_idx = -1

    def forward(self, words, dropout=0.1, scale=None):
        if dropout:
            size = (self.emb.weight.size(0),1)
            mask = dropout_mask(self.emb.weight.data, size, dropout)
            masked_emb_weight = mask * self.emb.weight
        else: masked_emb_weight = self.emb.weight
        if scale: masked_emb_weight = scale * masked_emb_weight
        return F.embedding(words, masked_emb_weight, self.pad_idx, self.emb.max_norm,
                           self.emb.norm_type, self.emb.scale_grad_by_freq, self.emb.sparse)

In [39]:
enc = nn.Embedding(100,20, padding_idx=0)
enc_dp = EmbeddingDropout(enc)

In [40]:
x = torch.randint(0,100,(25,)).long()

In [41]:
enc_dp(x, dropout=0.5)

tensor([[ 0.0000,  0.0000, -0.0000,  0.0000, -0.0000, -0.0000,  0.0000,  0.0000,
         -0.0000, -0.0000, -0.0000, -0.0000, -0.0000,  0.0000, -0.0000,  0.0000,
          0.0000, -0.0000,  0.0000, -0.0000],
        [-0.3189,  0.7055,  2.0192, -0.0838, -3.3327, -4.4722,  0.2896,  1.0168,
          0.6239, -2.6876,  1.3680,  0.4996,  2.5052,  2.9411, -0.2990, -0.0361,
          2.1119, -0.7355, -0.2148, -1.7215],
        [ 0.0000,  0.0000,  0.0000, -0.0000,  0.0000, -0.0000, -0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.0000,
         -0.0000, -0.0000,  0.0000, -0.0000],
        [ 1.8477, -2.0849,  0.9408,  0.5138,  0.8742,  1.9576,  0.9980, -0.8298,
         -0.2256, -1.8078,  3.8295, -0.3109, -4.3678, -0.5426,  3.1133,  1.4192,
         -0.7573,  3.0773, -1.7632,  1.2837],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.0000, -0.0000, -0.0000,
          0.0000,  0.0000,  0.0000,  0.0000, -0.0000, -0.0000,  0.0000,  0.0000,
      

### 2. AWD-LSTM

In [41]:
def repackage_var(h):
    "Detaches h from its history."
    return h.detach() if type(h) == torch.Tensor else tuple(repackage_var(v) for v in h)

In [42]:
class RNNCore(nn.Module):
    "AWD-LSTM/QRNN inspired by https://arxiv.org/abs/1708.02182"

    initrange=0.1

    def __init__(self, vocab_sz, emb_sz, n_hid, n_layers, pad_token, bidir=False,
                 dropouth=0.3, dropouti=0.65, dropoute=0.1, wdrop=0.5, qrnn=False):
        
        super().__init__()
        self.bs,self.qrnn,self.ndir = 1, qrnn,(2 if bidir else 1)
        self.emb_sz,self.n_hid,self.n_layers,self.dropoute = emb_sz,n_hid,n_layers,dropoute
        self.encoder = nn.Embedding(vocab_sz, emb_sz, padding_idx=pad_token)
        self.dp_encoder = EmbeddingDropout(self.encoder)
        if self.qrnn:
            #Using QRNN requires cupy: https://github.com/cupy/cupy
            from .torchqrnn.qrnn import QRNNLayer
            self.rnns = [QRNNLayer(emb_sz if l == 0 else n_hid, (n_hid if l != n_layers - 1 else emb_sz)//self.ndir,
                save_prev_x=True, zoneout=0, window=2 if l == 0 else 1, output_gate=True) for l in range(n_layers)]
            if wdrop:
                for rnn in self.rnns:
                    rnn.linear = WeightDropout(rnn.linear, wdrop, layer_names=['weight'])
        else:
            self.rnns = [nn.LSTM(emb_sz if l == 0 else n_hid, (n_hid if l != n_layers - 1 else emb_sz)//self.ndir,
                1, bidirectional=bidir) for l in range(n_layers)]
            if wdrop: self.rnns = [WeightDropout(rnn, wdrop) for rnn in self.rnns]
        self.rnns = torch.nn.ModuleList(self.rnns)
        self.encoder.weight.data.uniform_(-self.initrange, self.initrange)
        self.dropouti = RNNDropout(dropouti)
        self.dropouths = nn.ModuleList([RNNDropout(dropouth) for l in range(n_layers)])

    def forward(self, input):
        sl,bs = input.size()
        if bs!=self.bs:
            self.bs=bs
            self.reset()
        emb = self.dp_encoder(input, dropout=self.dropoute if self.training else 0)
        emb = self.dropouti(emb)
        raw_output = emb
        new_hidden,raw_outputs,outputs = [],[],[]
        for l, (rnn,drop) in enumerate(zip(self.rnns, self.dropouths)):
            #with warnings.catch_warnings():
            #    warnings.simplefilter("ignore")
            raw_output, new_h = rnn(raw_output, self.hidden[l])
            new_hidden.append(new_h)
            raw_outputs.append(raw_output)
            if l != self.n_layers - 1: raw_output = drop(raw_output)
            outputs.append(raw_output)
        self.hidden = repackage_var(new_hidden)
        return raw_outputs, outputs

    def one_hidden(self, l):
        nh = (self.n_hid if l != self.n_layers - 1 else self.emb_sz)//self.ndir
        return self.weights.new(self.ndir, self.bs, nh).zero_()

    def reset(self):
        [r.reset() for r in self.rnns if hasattr(r, 'reset')]
        self.weights = next(self.parameters()).data
        if self.qrnn: self.hidden = [self.one_hidden(l) for l in range(self.n_layers)]
        else: self.hidden = [(self.one_hidden(l), self.one_hidden(l)) for l in range(self.n_layers)]

In [43]:
class LinearDecoder(nn.Module):
    "To go on top of a RNN_Core module"
    
    initrange=0.1
    
    def __init__(self, n_out, n_hid, dropout, tie_encoder=None, bias=True):
        super().__init__()
        self.decoder = nn.Linear(n_hid, n_out, bias=bias)
        self.decoder.weight.data.uniform_(-self.initrange, self.initrange)
        self.dropout = RNNDropout(dropout)
        if bias: self.decoder.bias.data.zero_()
        if tie_encoder: self.decoder.weight = tie_encoder.weight

    def forward(self, input):
        raw_outputs, outputs = input
        output = self.dropout(outputs[-1])
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded, raw_outputs, outputs

In [44]:
class SequentialRNN(nn.Sequential):
    def reset(self):
        for c in self.children():
            if hasattr(c, 'reset'): c.reset()

In [45]:
def get_language_model(vocab_sz, emb_sz, n_hid, n_layers, pad_token, tie_weights=True, qrnn=False, bias=True,
                 dropout=0.4, dropouth=0.3, dropouti=0.5, dropoute=0.1, wdrop=0.5):
    "To create a full AWD-LSTM"
    rnn_enc = RNNCore(vocab_sz, emb_sz, n_hid=n_hid, n_layers=n_layers, pad_token=pad_token, qrnn=qrnn,
                 dropouth=dropouth, dropouti=dropouti, dropoute=dropoute, wdrop=wdrop)
    enc = rnn_enc.encoder if tie_weights else None
    return SequentialRNN(rnn_enc, LinearDecoder(vocab_sz, emb_sz, dropout, tie_encoder=enc, bias=bias))

In [46]:
tst_model = get_language_model(500, 20, 100, 2, 0, wdrop=0.)

In [47]:
x = torch.randint(0, 500, (10,5)).long()
z = tst_model(x)

In [48]:
len(z)

3

### 3. Callbacks to train the model

In [49]:
@dataclass
class RNNTrainer(Callback):
    learn:Learner
    bptt:int
    clip:float=None
    alpha:float=0.
    beta:float=0.
    
    def on_loss_begin(self, last_output, **kwargs):
        #Save the extra outputs for later and only returns the true output.
        self.raw_out,self.out = last_output[1],last_output[2]
        return last_output[0]
    
    def on_backward_begin(self, last_loss, last_input, last_output, **kwargs):
        #Adjusts the lr to the bptt selected
        self.learn.opt.lr *= last_input.size(0) / self.bptt
        #AR and TAR
        if self.alpha != 0.:  last_loss += (self.alpha * self.out[-1].pow(2).mean()).sum()
        if self.beta != 0.:
            h = self.raw_out[-1]
            if len(h)>1: last_loss += (self.beta * (h[1:] - h[:-1]).pow(2).mean()).sum()
        return last_loss
    
    def on_backward_end(self, **kwargs):
        if self.clip:  nn.utils.clip_grad_norm_(learn.model.parameters(), self.clip)

In [50]:
emb_sz, nh, nl = 400, 1150, 3
model = get_language_model(vocab_size, emb_sz, nh, nl, 0, dropouti=0.6, dropout=0.4, wdrop=0.5, 
                           dropoute=0.1, dropouth=0.2)
learn = Learner(data, model)

In [51]:
learn.opt_fn = partial(optim.Adam, betas=(0.8,0.99))
learn.callbacks.append(RNNTrainer(learn, bptt, alpha=2, beta=1))

In [52]:
fit_one_cycle(learn, 5e-3, 90, (0.8,0.7), wd=1.2e-6)

/home/ubuntu/anaconda3/envs/fastai1/lib/python3.7/site-packages/torch/nn/modules/module.py:477: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
  result = self.forward(*input, **kwargs)


0 6.806150378235847 6.421370460644777


1 6.379191435361401 6.017022414841538


2 6.104401945737696 5.754598051522358


3 5.881296827651156 5.550873076278194


...

85 3.8734474109072576 4.490099950146129


86 3.898653256934139 4.489182526404196


87 3.8427052566519846 4.483401919787831


88 3.863877234546905 4.479848650559775


89 3.938677174450208 4.474968436344681

